In [ ]:
# 经典题 四种写法 线段树 数组形式 动态开点懒更新节点形式 动态开点节点形式 求区间最值得反过来的树状数组

In [ ]:
class Solution:

    def maximumSumQueries(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        a = sorted(zip(nums1, nums2), key=lambda x: x[0])
        L, R = min(*nums2, *(y for _, y in queries)), max(*nums2, *(y for _, y in queries))
        # L, R = 1, 10**9
        mx = defaultdict(lambda: -1)  # [-1] * (2 << (R - L + 1).bit_length())  # 区间最大值 用defaultdict相当于动态开点了

        def update(o, l, r, idx, val):  # 单点设置
            if l == r:
                # 相同l 取最大 或只更新前面一次
                mx[o] = max(mx[o], val)  # if mx[o] == -1: mx[o] = val
                return
            m = l + r >> 1
            if m >= idx: update(o * 2, l, m, idx, val)
            else: update(o * 2 + 1, m + 1, r, idx, val)
            mx[o] = max(mx[o * 2], mx[o * 2 + 1])

        def query(o, l, r, si):  # 查询[si:]最大值
            if si <= l:
                return mx[o]
            m = l + r >> 1
            ans = query(o * 2 + 1, m + 1, r, si)
            if m >= si: ans = max(ans, query(o * 2, l, m, si))
            return ans

        ans = [-1] * len(queries)
        j = n - 1
        for i, (x, y) in sorted(enumerate(queries), key=lambda x: -x[1][0]):
            while j >= 0 and a[j][0] >= x:
                update(1, L, R, a[j][1], sum(a[j]))
                j -= 1
            ans[i] = query(1, L, R, y)
        return ans

In [ ]:
# 本题单点更新 无需lazy懒更新 只需要动态开点即可 看下一个题解去掉lazy 保留pushdown
class Node:
    __slots__ = 'si', 'ei', 'mi', 'left', 'right', 'mx', 'lazy'

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.mx = -1  # 区间最大值
        self.lazy = None  # 已经设置未下发


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def pushdown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)
        if node.lazy is None: return
        node.left.mx = node.right = node.left.lazy = node.right.lazy = node.lazy
        node.lazy = None

    def update(self, node: Node, idx, val):  # 单点更新
        if node.si == node.ei:
            node.mx = max(node.mx, val)
            return
        self.pushdown(node)
        if node.mi >= idx: self.update(node.left, idx, val)
        else: self.update(node.right, idx, val)
        node.mx = max(node.left.mx, node.right.mx)

    def query(self, node: Node, si):  # 查询[si:]
        if si <= node.si:
            return node.mx
        self.pushdown(node)
        ans = self.query(node.right, si)
        if node.mi >= si: ans = max(ans, self.query(node.left, si))
        return ans


class Solution:

    def maximumSumQueries(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        a = sorted(zip(nums1, nums2), key=lambda x: x[0])
        L, R = min(*nums2, *(y for _, y in queries)), max(*nums2, *(y for _, y in queries))
        # L, R = 1, 10**9
        st = SegmentTree(L, R)
        ans = [-1] * len(queries)
        j = n - 1
        for i, (x, y) in sorted(enumerate(queries), key=lambda x: -x[1][0]):
            while j >= 0 and a[j][0] >= x:
                st.update(st.root, a[j][1], sum(a[j]))
                j -= 1
            ans[i] = st.query(st.root, y)
        return ans

In [ ]:
# python 是一门动态语言，对于每一个类来说，我们可以在运行的过程中修改每个实例的属性和方法。
# 如果我们不想实例属性被改变，使用固定属性，这是可以使用__slots__属性去实现。
# __slots__ 可以减少内存开销
class Node:
    __slots__ = 'si', 'ei', 'mi', 'left', 'right', 'mx'

    def __init__(self, si, ei) -> None:
        self.si = si
        self.ei = ei
        self.mi = si + ei >> 1
        self.left = None
        self.right = None
        self.mx = -1  # 区间最大值


class SegmentTree:

    def __init__(self, si, ei) -> None:
        self.root = Node(si, ei)

    def pushdown(self, node: Node):
        if not node.left: node.left = Node(node.si, node.mi)
        if not node.right: node.right = Node(node.mi + 1, node.ei)

    def update(self, node: Node, idx, val):  # 单点更新
        if node.si == node.ei:
            node.mx = max(node.mx, val)
            return
        self.pushdown(node)
        if node.mi >= idx: self.update(node.left, idx, val)
        else: self.update(node.right, idx, val)
        node.mx = max(node.left.mx, node.right.mx)

    def query(self, node: Node, si):  # 查询[si:]
        if si <= node.si:
            return node.mx
        self.pushdown(node)
        ans = self.query(node.right, si)
        if node.mi >= si: ans = max(ans, self.query(node.left, si))
        return ans


class Solution:

    def maximumSumQueries(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        n = len(nums1)
        a = sorted(zip(nums1, nums2), key=lambda x: x[0])
        L, R = min(*nums2, *(y for _, y in queries)), max(*nums2, *(y for _, y in queries))
        # L, R = 1, 10**9
        st = SegmentTree(L, R)
        ans = [-1] * len(queries)
        j = n - 1
        for i, (x, y) in sorted(enumerate(queries), key=lambda x: -x[1][0]):
            while j >= 0 and a[j][0] >= x:
                st.update(st.root, a[j][1], sum(a[j]))
                j -= 1
            ans[i] = st.query(st.root, y)
        return ans

In [ ]:
class Solution:

    def maximumSumQueries(self, nums1: List[int], nums2: List[int], queries: List[List[int]]) -> List[int]:
        # 离散化
        lst = sorted(set((*nums2, *(y for _, y in queries))))
        n = len(lst)
        mx = [-1] * (n + 1)  # 区间最大值

        def update(x, val):  # 单点设置val
            while x > 0:  # 正常是 x <= n 但是树状数组区间查询只能是[1,x] 而我们要查[x,n]所以要在线段上反过来单点设置
                mx[x] = max(mx[x], val)  # 如果是区间和 得+= 此处+表示区间和 累加
                x -= x & -x

        def query(x):  # 查询[x:]最大值
            ans = -1
            while x <= n:  # 正常是 x > 0 但是树状数组区间查询只能是[1,x] 而我们要查[x,n]所以要在线段上反过来单点设置
                ans = max(ans, mx[x])
                x += x & -x
            return ans

        a = sorted(zip(nums1, nums2), key=lambda x: x[0])
        ans = [-1] * len(queries)
        j = len(nums1) - 1
        for i, (x, y) in sorted(enumerate(queries), key=lambda x: -x[1][0]):
            while j >= 0 and a[j][0] >= x:
                update(bisect_left(lst, a[j][1]) + 1, sum(a[j]))
                j -= 1
            ans[i] = query(bisect_left(lst, y) + 1)
        return ans

In [10]:
a = [8,9]
b = [[1,7],[2,10]]
min(*a, (y for _, y in b)),(*a, *(y for _, y in b))

([7, 10], (8, 9, 7, 10))